# Setup

In [1]:
from setup import *

In [2]:
from pqdm.processes import pqdm

In [3]:
import pqdm

In [3]:
aus = City('Austin, TX')
cam = City('Cambridge, MA')
tor = City('City of Toronto', 'Toronto')

In [4]:
cities = [cam, tor, aus]

---
# Create pednet variants

In [5]:
od_fname = 'sample_OD_n1000000_seed1234_d800m'

In [6]:
%%time
# t = 22s
pns = [Pednet(*params, od_fname) for params in tqdm([
    # Cambridge
    (cam, 'exists', 'current'),
    (cam, 'exists or is_xwalk', 'crosswalks'),
    # Austin
    (aus, 'exists', 'current'),
    (aus, 'exists or ~is_xwalk', 'sidewalks'),
    (aus, 'exists or is_xwalk', 'crosswalks'),
    (aus, None, 'both'),
    # Toronto
    (tor, 'exists', 'current'),
    (tor, 'exists or ~is_xwalk', 'sidewalks'),
    (tor, 'exists or is_xwalk', 'crosswalks'),
    (tor, None, 'both'),
])]
for c in cities:
    c.pn = {pn.name: pn for pn in pns if pn.city == c}

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 16.5 s, sys: 1.39 s, total: 17.9 s
Wall time: 20.1 s


## Get edge betweenness

In [7]:
def get_edge_bwness(pn):
    pn.E['bwness'] = pn.G.edge_betweenness(False, weights=pn.E['len'])
    pn.E.to_pickle(U.mkdir(pn.city.root + 'edges/') + f'{pn.name}.pickle')
    
# %time x = get_edge_bwness(pns[4]); x

In [8]:
# total time: 8h 10min
# {Cambridge: 20s, Austin: 4h 51min + some more time, Toronto: 1h 31min + some more time}
# %time for c in tqdm([cam, aus, tor]): print(c); get_edge_bwness(c)

In [9]:
for pn in pns:
    pn.E = pd.read_pickle(pn.city.root + f'edges/{pn.name}.pickle')

---
# Get shortest paths

## Define shortest path functions for `pqdm`

### New (per pednet)

In [10]:
func_template = 'def sp_{}_{}(o, d): return pns[{}].get_sp(o, d)'
for i, pn in enumerate(pns):
    exec(func_template.format(pn.city.name, pn.name, i))

## Compute shortest paths

In [11]:
def get_shortest_path(pn, n_jobs=30, save=True):
    # filter unique connected OD pairs
    od = (pn.od.groupby(['vid_o','vid_d'])['n_ods'].sum().reset_index()
          .merge(pn.V['cid'].rename('cid_o'), left_on='vid_o', right_index=True)
          .merge(pn.V['cid'].rename('cid_d'), left_on='vid_d', right_index=True)
          .query('cid_o == cid_d').drop(columns=['cid_o','cid_d']))
    func = globals()[f'sp_{pn.city.name}_{pn.name}']
    sp = pqdm(zip(od['vid_o'], od['vid_d']), func, n_jobs=n_jobs,
              total=od.shape[0], argument_type='args')
    try:
        sp = (od.merge(Pdf(sp), on=('vid_o','vid_d'))
              .sort_values(['vid_o','vid_d']).reset_index(drop=True))
        if save:
            outdir = U.mkdir(pn.city.root + 'paths/')
            sp.to_pickle(outdir + '{}_{}od.pickle'.format(
                pn.name, pn.od['n_ods'].sum()))
    except Exception as e:
        print(e)
        return sp
    return sp

# %time x = get_shortest_path(tor.pn[0]); x

## Compute edge weight (volume)

In [12]:
def get_edge_weights(pn):
    # od = pn.od.groupby(['vid_o','vid_d'])['n_ods'].sum().reset_index()
    E = (pd.read_pickle(glob(pn.city.root + f'edges/*{pn.name.lower()}.pickle')[0])
         .drop(columns='n_trips', errors='ignore'))
    pn.sp = pd.read_pickle(glob(pn.city.root + f'paths/{pn.name}*.pickle')[0])
    sp = (pn.sp.set_index('n_ods')['edges'].rename('id').explode().reset_index()
          .groupby('id')['n_ods'].sum().rename('n_trips').reset_index())
    E = E.merge(sp, on='id', how='left').fillna({'n_trips': 0})
    pn.E = E.astype({'n_trips': int})
    
# %time x = get_edge_weights(pns[4]); x

## Pednet score

## Export edges